<a href="https://colab.research.google.com/github/yaraeslamm/Video_Summarization/blob/main/VideoSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from datetime import timedelta

# Helper function to draw timestamp on image
def draw_timestamp(image, timestamp, x, y):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_color = (100, 9, 198)
    thickness = 2
    cv2.putText(image, timestamp, (x, y), font, font_scale, font_color, thickness, lineType=cv2.LINE_AA)

# Background Modeling using median frame
def create_background_model(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        print("Error: Could not open video file.")
        return None

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray_frame)
    cap.release()

    if len(frames) == 0:
        print("Error: No frames read from video.")
        return None

    # Create the reference frame as the median of all frames
    median_frame = np.median(frames, axis=0).astype(dtype=np.uint8)
    return median_frame

# Detect objects using frame differencing
def detect_objects(frame, background):
    diff = cv2.absdiff(frame, background)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Morphological operations to remove noise and fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return [cnt for cnt in contours if cv2.contourArea(cnt) > 500]  # Adjust MIN_AREA if necessary

# Euclidean distance tracker
class EuclideanTracker:
    def __init__(self):
        self.center_points = {}
        self.id_count = 0
        self.extracted_ids = set()  # Track IDs of already extracted objects

    def update(self, objects_rect):
        objects_ids = []
        for rect in objects_rect:
            x, y, w, h = rect
            cx = x + w // 2
            cy = y + h // 2

            # Find if this object is close to any existing points
            same_object = False
            for id, point in list(self.center_points.items()):
                dist = np.hypot(point[0] - cx, point[1] - cy)

                if dist < 90:  # Increased threshold for larger objects (like vehicles)
                    self.center_points[id] = (cx, cy)
                    objects_ids.append((id, rect))
                    same_object = True
                    break

            if not same_object:
                self.center_points[self.id_count] = (cx, cy)
                objects_ids.append((self.id_count, rect))
                self.id_count += 1

        # Remove old IDs not found in this frame
        self.center_points = {k: v for k, v in self.center_points.items() if k in [o[0] for o in objects_ids]}

        return objects_ids

# Main pipeline
def process_video(video_path, output_dir):
    # Create background model and prepare video capture
    background = create_background_model(video_path)
    if background is None:
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    tracker = EuclideanTracker()
    frame_count = 0

    # Create reference frame: background model as a 3-channel (BGR) image
    reference_frame = np.stack([background] * 3, axis=-1)  # Convert gray to BGR
    object_positions = []  # To keep track of the objects in the reference frame
    max_width = reference_frame.shape[1] + 20
    current_x = 50  # Starting X coordinate for placing objects in the reference frame
    current_y = 50  # Starting Y coordinate for placing objects

    # Process each frame in the video
    while True:
        ret, frame = cap.read()
        if not ret:
            print("No more frames to process.")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        contours = detect_objects(gray_frame, background)

        objects_rect = [cv2.boundingRect(cnt) for cnt in contours]
        tracked_objects = tracker.update(objects_rect)

        # Draw bounding boxes for the detected objects and add timestamp
        for obj_id, rect in tracked_objects:
            if obj_id in tracker.extracted_ids:
                continue  # Skip already extracted objects

            tracker.extracted_ids.add(obj_id)  # Mark this object as extracted
            x, y, w, h = rect
            cropped_object = frame[y:y + h, x:x + w]

            # Calculate timestamp with milliseconds
            timestamp = str(timedelta(seconds=frame_count / cap.get(cv2.CAP_PROP_FPS)))  # Convert frame count to timestamp
            timestamp = timestamp[:-3]  # Truncate to 10th of millisecond
            draw_timestamp(reference_frame, timestamp, current_x, current_y - 10)  # Draw timestamp near the object

            # Check if we have enough space horizontally to add the next object
            if current_x + w + 50 > max_width:  # If no space, move to the next row
                current_x = 50
                current_y += h + 30  # Add some space between rows

            # Check if we exceed the max limit of objects per frame
            if len(object_positions) >= 10:
                break

            if current_y + cropped_object.shape[0] > reference_frame.shape[0]:
                # If not enough space, resize cropped_object to fit
                cropped_object = cv2.resize(cropped_object, (cropped_object.shape[1], reference_frame.shape[0] - current_y))

            reference_frame[current_y:current_y + cropped_object.shape[0], current_x:current_x + cropped_object.shape[1]] = cropped_object

            # Update the current x position for the next object
            current_x += w + 50  # Add some horizontal space between objects
            object_positions.append((current_x, current_y))

        frame_count += 1

    cap.release()

    # Save the final reference frame containing objects and timestamps
    cv2.imwrite(f"{output_dir}/reference_frame_with_objects.png", reference_frame)
    print(f"Reference frame with objects saved as {output_dir}/reference_frame_with_objects.png")

    # Save output frames (with detected objects and bounding boxes)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        contours = detect_objects(gray_frame, background)

        objects_rect = [cv2.boundingRect(cnt) for cnt in contours]
        tracked_objects = tracker.update(objects_rect)

        # Draw bounding boxes for the detected objects and add timestamp
        for obj_id, rect in tracked_objects:
            x, y, w, h = rect
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw rectangle around detected object

            # Calculate timestamp with milliseconds
            timestamp = str(timedelta(seconds=frame_count / cap.get(cv2.CAP_PROP_FPS)))  # Convert frame count to timestamp
            timestamp = timestamp[:-3]  # Truncate to 10th of millisecond
            draw_timestamp(frame, timestamp, x, y - 10)  # Draw timestamp near the object

        # Save the output frame with detected objects
        frame_output_path = f"{output_dir}/frame_{frame_count:04d}_detected.png"
        if cv2.imwrite(frame_output_path, frame):
            print(f"Saved: {frame_output_path}")
        else:
            print(f"Error saving: {frame_output_path}")

        frame_count += 1

    cap.release()
    print("Processing complete.")

# Ensure the output directory exists
OUTPUT_DIR = '/content/drive/My Drive/AVP/Projects'  # Change this as needed
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Video path
VIDEO_PATH = '/content/drive/My Drive/RuskaUfer25A.avi'  # Update with your video path

# Run the function
process_video(VIDEO_PATH, OUTPUT_DIR)



No more frames to process.
Reference frame with objects saved as /content/drive/My Drive/AVP/Projects/reference_frame_with_objects.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0000_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0001_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0002_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0003_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0004_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0005_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0006_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0007_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0008_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0009_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0010_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame_0011_detected.png
Saved: /content/drive/My Drive/AVP/Projects/frame